In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import time

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)

driver.get("https://www.instagram.com/accounts/login/")
print("수동으로 로그인 후 40초 안에 인증을 완료해주세요")
time.sleep(40)

with open("instagram_cookies.pkl", "wb") as f :
    pickle.dump(driver.get_cookies(), f)

print("쿠키 저장 완료")
driver.quit()

수동으로 로그인 후 40초 안에 인증을 완료해주세요
쿠키 저장 완료


In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import pickle
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 12)

driver.get("https://www.instagram.com/")
with open("instagram_cookies.pkl", "rb") as f :
    cookies = pickle.load(f)
for cookie in cookies :
    driver.add_cookie(cookie)

driver.refresh()
time.sleep(5)

try :
    not_now = driver.find_element(By.XPATH, '//button[text()="나중에 하기"]')
    not_now.click()
    time.sleep(2)
except :
    pass

print("정상적인 로그인 후 나중에하기 버튼 클릭 완료")

keyword = "올리브영"
driver.get(f"https://www.instagram.com/explore/tags/{keyword}/")
time.sleep(5)

# 올리브영이라는 검색어를 입력한 이후에 처음보여지는 화면에서 a태그를 9개를 찾아올 예정
# 9 => 9 * 10 => 90개 a태그를 찾아올 확률 
# 90개 => ['a href="https://www.instagram.com/p/DNWz-KOzvN1/"', 'a href="https://www.instagram.com/p/DNWz-KOzvN1/"', 'a href="https://www.instagram.com/p/DNWz-KOzvN1/"', ...]
# post_links = ["https://www.instagram.com/p/DNWz-KOzvN1/", "https://www.instagram.com/p/DNWz-KOzvN1/", "https://www.instagram.com/p/DNWz-KOzvN1/")

post_links = set()
for _ in range(3) :
    links = driver.find_elements(By.TAG_NAME, "a")
    for link in links :
        href = link.get_attribute("href")
        if href and "/p/" in href :
            post_links.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

# 90개의 URL
# 1URL => 0번
# 2URL => 1번

results = []
for url in list(post_links)[0:5] :
    driver.get(url)
    time.sleep(3)
    txt = ""

    try :
        # h1 = driver.find_element(By.XPATH, '//h1[@dir="auto"]')
        # txt = h1.text.strip()
        h1 = wait.until(EC.presence_of_element_located((By.XPATH, "//article//h1[@dir='auto']")))
        txt = h1.get_attribute("textContent").strip()
    except :
        pass

    if not txt :
        try :
            img_tag = driver.find_element(By.XPATH, "//img[@alt]")
            txt = img_tag.get_attribute("alt").strip()
        except Exception as e :
            print(f"txt 추출 실패 : {url} | 에러 : {e}")
            continue
    if txt :
        results.append({"url": url, "txt": txt})

driver.quit()

for r in results :
    print(r["url"])
    print(r["txt"])
    print("="*60)

정상적인 로그인 후 나중에하기 버튼 클릭 완료
https://www.instagram.com/p/DKg1ZgLSMXc/
Photo by 행복한 일상★ on June 05, 2025. 헤어 제품, 핸드크림 및 로션의 이미지일 수 있음.
https://www.instagram.com/p/DNjoAA8MCzB/
Photo by Snp® 에스엔피 on August 19, 2025. 사람 1명, 메이크업, 핸드크림, 화장품 및 문구: 'BGOUATINGTONEA GLUTATHI EXFOLIATING EXFOLIATING CLEAR MASK TONE-UP TONE-UP ONEX GLUTATHIONEXPO PDRN 00/3 FLAWLESS VLESS /3.350 oли 3 Snp®'의 이미지일 수 있음.
https://www.instagram.com/p/DLJaBgBu30t/
Photo by 20대 뭐 하지? on June 20, 2025. 헤어 제품, 약품, 로션, 핸드크림, 연고 및 텍스트의 이미지일 수 있음.
https://www.instagram.com/p/DNoyF5ly7Mg/
Photo by 짤퀸 on August 21, 2025. 사람 6명 및 텍스트의 이미지일 수 있음.
https://www.instagram.com/p/DKO4VLGKkl_/
Photo by 20대 뭐 하지? on May 29, 2025. 사진 설명이 없습니다..


In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pickle, time, os
import pandas as pd
from datetime import datetime

# -----------------------------
# 1) 드라이버 세팅
# -----------------------------
service = Service(ChromeDriverManager().install())
options = Options()
# options.add_argument("--headless=new")  # 필요시 헤드리스
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--lang=ko_KR")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                     "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 12)  # ✅ 드라이버 생성 이후에 선언

# -----------------------------
# 2) 쿠키로 로그인 세션 주입
# -----------------------------
driver.get("https://www.instagram.com/")
with open("instagram_cookies.pkl", "rb") as f:
    cookies = pickle.load(f)
for c in cookies:
    try:
        driver.add_cookie(c)
    except:
        # 도메인/경로 이슈 있으면 무시 (새로고침 후 재시도로 해결되는 경우가 많음)
        pass

driver.refresh()
time.sleep(2)

# 팝업 닫기 (button/div 두 케이스)
for xp in [
    "//button[normalize-space()='나중에 하기']",
    "//div[@role='button' and contains(normalize-space(.), '나중에 하')]",
    "//div[@role='button' and contains(normalize-space(.), 'Not now')]",
]:
    try:
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, xp))).click()
        break
    except:
        pass

print("정상적인 로그인 후 나중에하기 버튼 처리 완료")

# -----------------------------
# 3) 해시태그 페이지에서 포스트 링크 수집
# -----------------------------
keyword = "올리브영"
driver.get(f"https://www.instagram.com/explore/tags/{keyword}/")
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[href*='/p/']")))

post_links = set()
for _ in range(3):
    for a in driver.find_elements(By.CSS_SELECTOR, "a[href*='/p/']"):
        href = a.get_attribute("href")
        if href:
            post_links.add(href)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

print(f"수집한 링크 수: {len(post_links)}")

# -----------------------------
# 4) 캡션 추출 유틸
# -----------------------------
def _inner_text(el):
    # .text 대신 innerText/textContent 사용
    try:
        txt = driver.execute_script("return arguments[0].innerText;", el) or ""
    except Exception:
        txt = el.get_attribute("textContent") or ""
    return txt.strip()

def extract_caption_safely():
    # 상세 페이지 로딩 신호: article 또는 h1/h2 또는 img[alt] 중 아무거나
    try:
        wait.until(EC.any_of(
            EC.presence_of_element_located((By.TAG_NAME, "article")),
            EC.presence_of_element_located((By.XPATH, "//h1|//h2")),
            EC.presence_of_element_located((By.XPATH, "//img[@alt]"))
        ))
    except Exception:
        # 어떤 이유로든 신호 못 잡으면 1초 대기 후 진행
        time.sleep(1)

    # 가능성 높은 후보들을 넓은 → 좁은 순서로 시도
    XPATHS = [
        "//article//h1[@dir='auto']",
        "//article//h1",
        "//article//*[self::h1 or self::h2][@dir]",
        "//article//div[contains(@class,'xt0psk2')]//h1",          # 스샷에서 보인 패턴
        "//article//div[contains(@class,'html-div')]",             # 자주 쓰이는 캡션 컨테이너
        "//h1[@dir='auto']",
        "//h1",
    ]
    for xp in XPATHS:
        try:
            el = driver.find_element(By.XPATH, xp)
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            txt = _inner_text(el)
            if txt:
                return txt
        except:
            pass

    # 최후: JS로 article 내부에서 가장 긴 가시 텍스트 블록 선택
    try:
        longest = driver.execute_script("""
            const art = document.querySelector('article');
            if (!art) return '';
            const sels = 'h1,h2,div[dir],span[dir],div,span';
            const nodes = Array.from(art.querySelectorAll(sels))
              .map(n => (n.innerText || '').trim())
              .filter(t => t && t.length >= 10);
            if (!nodes.length) return '';
            nodes.sort((a,b) => b.length - a.length);
            return nodes[0];
        """) or ""
        longest = (longest or "").strip()
        if longest:
            return longest
    except:
        pass

    # 정말 없으면 대표 이미지 alt
    try:
        img = driver.find_element(By.XPATH, "//article//img[@alt]")
        return img.get_attribute("alt").strip()
    except:
        return ""

# -----------------------------
# 5) 상세 페이지 순회 & 캡션 수집
# -----------------------------
results = []
for url in list(post_links)[:30]:  # 원하는 개수로 제한
    driver.get(url)
    caption = extract_caption_safely()
    if caption:
        results.append({"url": url, "caption": caption})
    else:
        print(f"[❌] 캡션 추출 실패: {url}")

driver.quit()

# -----------------------------
# 6) 결과 출력 + 엑셀 저장
# -----------------------------
for r in results:
    print(r["url"])
    print(r["caption"])
    print("=" * 60)

os.makedirs("output", exist_ok=True)
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
xlsx_path = f"output/instagram_{keyword}_{ts}.xlsx"
pd.DataFrame(results, columns=["url", "caption"]).to_excel(xlsx_path, index=False)
print(f"[✅] 엑셀 저장 완료: {xlsx_path}")
# CSV도 필요하면 아래 주석 해제
# pd.DataFrame(results).to_csv(f"output/instagram_{keyword}_{ts}.csv", index=False, encoding="utf-8-sig")


정상적인 로그인 후 나중에하기 버튼 처리 완료
수집한 링크 수: 39
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKO4VLGKkl_/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKTVXheSE2A/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNhs_08ywNj/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKRuWxJynAA/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKmMLPfzEeA/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNO348lyo1m/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKEDaj7yvyx/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNiZkrqpOjg/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DLzDRWAyXFR/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNsDoAMXOlT/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNsDpOZXpjM/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DK_bd8ovrEF/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKUYBzpTFEv/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNpyLwDC3tI/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DKeJcvKSy4Q/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNrdymkyTeI/
[❌] 캡션 추출 실패: https://www.instagram.com/p/DNp6P8jh21_/
[❌] 캡션 추출 실패: https://www.